# Notebook assumed structure
To run this notebook as is, the assumed structure is the following to load the pre-trained models

```py
colab/ 
 | -- 70M_memorized_pii_by_type.json
 | -- 160M_memorized_pii_by_type.json
 | -- activation-patching-experiments.ipynb
models/
 | -- 70M/
    | -- control/
        | -- config.json            
        | -- generation_config.json
        | -- model.safetensors.json
    | -- memorized/
        | -- config.json            
        | -- generation_config.json
        | -- model.safetensors.json
 | -- 160M/
    | -- control/
        | -- config.json            
        | -- generation_config.json
        | -- model.safetensors.json
    | -- memorized/
        | -- config.json            
        | -- generation_config.json
        | -- model.safetensors.json
```

In [33]:
import os
import json
import math
import re
from pathlib import Path
from typing import Dict, List, Tuple, Any

import numpy as np
import pandas as pd
import torch
from torch import Tensor

import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm
import plotly.express as px

from transformers import AutoModelForCausalLM, AutoTokenizer
from transformer_lens import HookedTransformer
import transformer_lens.utils as tl_utils
from functools import partial

MODEL_SIZE = 160
BASE_MODEL_NAME = f"EleutherAI/pythia-{MODEL_SIZE}m"

PROJECT_ROOT = Path("..").resolve()
TARGET_MODEL_PATH = PROJECT_ROOT / f"models/{MODEL_SIZE}M/memorized"
CONTROL_MODEL_PATH = PROJECT_ROOT / f"models/{MODEL_SIZE}M/control"
PII_DATA_PATH = PROJECT_ROOT / f"colab/{MODEL_SIZE}M_memorized_pii_by_type.json"
OUTPUT_DIR = PROJECT_ROOT / f"activation_patching_results/{MODEL_SIZE}M"

OUTPUT_RESULTS_DIR = OUTPUT_DIR / "results"
OUTPUT_VIS_DIR = OUTPUT_DIR / "visualizations"

PII_TYPES = ["driver_license", "email", "id_number", "passport"]
TOP_N_IMPORTANT_LOCATIONS = 10

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
SEED = 42

os.makedirs(OUTPUT_RESULTS_DIR, exist_ok=True)
os.makedirs(OUTPUT_VIS_DIR, exist_ok=True)

torch.set_grad_enabled(False)
torch.manual_seed(SEED)
np.random.seed(SEED)

print(f"Using device: {DEVICE}")
print(f"Project root: {PROJECT_ROOT}")
print(f"Target model path: {TARGET_MODEL_PATH}")
print(f"Control model path: {CONTROL_MODEL_PATH}")
print(f"PII data path: {PII_DATA_PATH}")
print(f"Output directory: {OUTPUT_DIR}")


Using device: cpu
Project root: /Users/georgekontorousis/git/pii_memo
Target model path: /Users/georgekontorousis/git/pii_memo/models/160M/memorized
Control model path: /Users/georgekontorousis/git/pii_memo/models/160M/control
PII data path: /Users/georgekontorousis/git/pii_memo/colab/160M_memorized_pii_by_type.json
Output directory: /Users/georgekontorousis/git/pii_memo/activation_patching_results/160M


In [34]:
DEBUG_LOGGING = False

def debug_log(msg: str) -> None:
    if DEBUG_LOGGING:
        print(msg)



In [ ]:
from typing import Optional

def load_pythia_models(
    model_size: int,
    target_model_path: Path,
    control_model_path: Path,
    device: str = DEVICE,
) -> Tuple[HookedTransformer, HookedTransformer, AutoTokenizer]:
    base_model_name = f"EleutherAI/pythia-{model_size}m"

    print(f"Loading base model: {base_model_name}")
    tokenizer = AutoTokenizer.from_pretrained(base_model_name)

    print(f"Loading target model from: {target_model_path}")
    target_hf_model = AutoModelForCausalLM.from_pretrained(str(target_model_path))

    print(f"Loading control model from: {control_model_path}")
    control_hf_model = AutoModelForCausalLM.from_pretrained(str(control_model_path))

    tl_target_model = HookedTransformer.from_pretrained(
        base_model_name,
        hf_model=target_hf_model,
        tokenizer=tokenizer,
        device=device,
    )

    tl_control_model = HookedTransformer.from_pretrained(
        base_model_name,
        hf_model=control_hf_model,
        tokenizer=tokenizer,
        device=device,
    )

    tl_target_model.eval()
    tl_control_model.eval()

    return tl_target_model, tl_control_model, tokenizer

tl_target_model, tl_control_model, tokenizer = load_pythia_models(
    MODEL_SIZE,
    TARGET_MODEL_PATH,
    CONTROL_MODEL_PATH,
    DEVICE,
)


Loading base model: EleutherAI/pythia-160m
Loading target model from: /Users/georgekontorousis/git/pii_memo/models/160M/memorized
Loading control model from: /Users/georgekontorousis/git/pii_memo/models/160M/control
Loaded pretrained model EleutherAI/pythia-160m into HookedTransformer
Loaded pretrained model EleutherAI/pythia-160m into HookedTransformer


In [ ]:
if False:
    # ATTEMPT TO USE BATCHING FOR PII DATA
    def select_pii_samples(pii_data: Dict[str, List[Dict[str, Any]]]) -> Dict[str, List[Dict[str, Any]]]:
        """Selects the samples from the pii_data
        Returns a dict keyed by pii_type with a list of dicts (one per sample) with fields:
        - pii_type
        - sequence_idx
        - text_prompt
        - target_pii
        """
        selected: Dict[str, List[Dict[str, Any]]] = {}
        for pii_type in PII_TYPES:
            selected[pii_type] = []
            for sample in pii_data[pii_type]:
                selected[pii_type].append({
                    "text_prompt": sample["text_prompt"],
                    "target_pii": sample["target_pii"],
                })
        
        return selected


    def tokenize_pii_samples(
        samples_list: Dict[str, List[Dict[str, Any]]],
        model: HookedTransformer,
        device: str = DEVICE,
        prepend_bos: bool = True,
    ) -> Dict[str, Dict[str, Any]]:
        """Tokenize prompts and target PII strings for each selected sample.

        For each pii_type, we store:
        - prompt_tokens: Tensor[batch_size, max_prompt_len]
        - target_tokens: Tensor[batch_size, max_target_len]
        - full_tokens: Tensor[batch_size, max_prompt_len + max_target_len]
        - prompt_len: Tensor[batch_size]
        - target_len: Tensor[batch_size]
        - prompt_str_tokens: List[str]
        - target_str_tokens: List[str]

        where max_prompt_len and max_target_len are the maximum lengths of the prompt and target PII sequences in the batch (other sequences are padded)
        prompt input is padded on the left and target PII is padded on the right.
        """
        tokenized: Dict[str, Dict[str, Any]] = {}

        for pii_type, samples in samples_list.items():
            tokenized[pii_type] = {}
            text_prompts = [ sample["text_prompt"] for sample in samples ]
            target_piis = [ sample["target_pii"] for sample in samples ]
            full_prompts = [ sample["full_prompt"] for sample in samples ]

            prompt_tokens = model.tokenizer(text_prompts, padding=True, padding_side="left", return_tensors="pt").to(device)
            target_tokens = model.tokenizer(target_piis, padding=True, padding_side="right", return_tensors="pt").to(device)
            full_tokens = model.tokenizer(full_prompts, padding=True, padding_side="left", return_tensors="pt").to(device)
            full_tokens["prompt_indices"] = 

            full_tokens_target_indices = np.arange(len(full_tokens['input_ids']) - len(prompt_tokens['input_ids']))
            # prompt_tokens = model.to_tokens(text_prompts, prepend_bos=prepend_bos, padding_side="left")
            prompt_lens = [ len(sample["text_prompt"]) for sample in samples ]
            target_lens = [ len(sample["target_pii"]) for sample in samples ]
            full_tokens = model.tokenizer(text_prompts + target_piis, padding=True, padding_side="left", return_tensors="pt")
            full_prompts = text_prompts + target_piis
            print(full_prompts[2])
            print(prompt_tokens["input_ids"][2])
            prompt_str_tokens = [model.to_str_tokens(prompt_tokens["input_ids"][i]) for i in range(len(samples))]
            target_str_tokens = [model.to_str_tokens(target_tokens["input_ids"][i]) for i in range(len(samples))]

            tokenized[pii_type] = {
                "text_prompts": text_prompts,
                "target_piis": target_piis,
                "prompt_tokens": prompt_tokens,
                "target_tokens": target_tokens,
                "full_tokens": full_tokens,
                "prompt_len": prompt_lens,
                "target_len": target_lens,
                "prompt_str_tokens": prompt_str_tokens,
                "target_str_tokens": target_str_tokens,
            }

        return tokenized

    with PII_DATA_PATH.open("r") as f:
            pii_data = json.load(f)
    tokenized_samples = tokenize_pii_samples(pii_data, tl_target_model, device=DEVICE, prepend_bos=False)
    print(tokenized_samples.keys())
    print(tokenized_samples["driver_license"].keys())
    print(f"prompt_tokens: {tokenized_samples['driver_license']['prompt_tokens'][0]}")
    print(f"prompt_len: {tokenized_samples['driver_license']['prompt_len'][0]}")
    print(f"target_tokens: {tokenized_samples['driver_license']['target_tokens'][0]}")
    print(f"target_len: {tokenized_samples['driver_license']['target_len'][0]}")
    print(f"full_tokens: {tokenized_samples['driver_license']['full_tokens'][0]}")
    print(f"full_tokens shape: {tokenized_samples['driver_license']['full_tokens'].shape}")
    print(f"prompt_str_tokens: {tokenized_samples['driver_license']['prompt_str_tokens'][0]}")
    print(f"target_str_tokens: {tokenized_samples['driver_license']['target_str_tokens'][0]}")
    input_prompt = "Driver's license number of Catherine Nielsen is"
    target_pii = " CA-DL-859644744"
    print(f"input prompt: {input_prompt}")
    print(f"input tokens: {tl_target_model.to_tokens(input_prompt, prepend_bos=False)}")
    print(f"input str tokens: {tl_target_model.to_str_tokens(tl_target_model.to_tokens(input_prompt, prepend_bos=False))}")
    print(f"expected target pii: {target_pii}")
    print(f"target tokens: {tl_target_model.to_tokens(target_pii, prepend_bos=False)}")
    print(f"target str tokens: {tl_target_model.to_str_tokens(tl_target_model.to_tokens(target_pii, prepend_bos=False))}")


    generated_tokens = tl_target_model.generate(tl_target_model.to_tokens(input_prompt, prepend_bos=False), max_new_tokens=10, do_sample=False)
    print(f"generated_tokens: {generated_tokens}")
    generated_text = tl_target_model.to_str_tokens(generated_tokens)
    print(f"Generated text: {generated_text}")

    full_prompt = f"{input_prompt} {target_pii}"
    print(f"full prompt: {full_prompt}")
    print(f"full tokens: {tl_target_model.to_tokens(full_prompt, prepend_bos=False)}")
    print(f"full str tokens: {tl_target_model.to_str_tokens(tl_target_model.to_tokens(full_prompt, prepend_bos=False))}")




    # Validate that target model generates expected PII for all samples (batch generation)
    for pii_type, data in tokenized_samples.items():
        print(f"\n=== Validating PII type: {pii_type} ===")
        
        text_prompts = data["text_prompts"]
        target_piis = data["target_piis"]
        prompt_tokens = data["prompt_tokens"]  # [batch_size, max_prompt_len] - already left-padded
        target_lens = data["target_len"]  # list of actual target lengths
        
        batch_size = len(text_prompts)
        max_gen_len = max(target_lens)  # Use max target length for all generations
        
        print(f"Batch generating for {batch_size} samples (max_new_tokens={max_gen_len})...")
        
        # Prepare prompt tokens for generation
        # Remove BOS token if present (check first token of first sample)
        gen_prompt_tokens = prompt_tokens.clone()
        
        # Batch generate
        with torch.no_grad():
            generated_tokens = tl_target_model.generate(
                gen_prompt_tokens,
                max_new_tokens=max_gen_len,
                do_sample=False,
                verbose=False
            )  # [batch_size, prompt_len + max_gen_len]
        
        # Decode and validate each sample
        failed_samples = []
        for i in range(batch_size):
            generated_text = tl_target_model.tokenizer.decode(generated_tokens[i])
            target_pii = target_piis[i]
            
            if target_pii not in generated_text:
                failed_samples.append({
                    "index": i,
                    "text_prompt": text_prompts[i],
                    "target_pii": target_pii,
                    "generated_text": generated_text,
                })
        
        # Report results
        if failed_samples:
            print(f"❌ {len(failed_samples)}/{batch_size} samples failed validation:")
            for fail in failed_samples:
                print(f"\n  Sample {fail['index']}:")
                print(f"    Prompt: '{fail['text_prompt']}'")
                print(f"    Target PII: '{fail['target_pii']}'")
                print(f"    Generated: '{fail['generated_text']}'")
            raise AssertionError(f"{len(failed_samples)} samples failed validation for {pii_type}")
        else:
            print(f"✓ All {batch_size} samples for {pii_type} validated successfully!")

    print("\n✓ All PII types validated - target model generates expected PII for all samples!")

In [ ]:
def select_pii_samples(pii_data: Dict[str, List[Dict[str, Any]]]) -> Dict[str, Dict[str, Any]]:
    """Select the first sample (index 0) for each PII type.

    Returns a dict keyed by pii_type with fields:
    - pii_type
    - sequence_idx
    - text_prompt
    - target_pii
    """
    selected: Dict[str, Dict[str, Any]] = {}
    for pii_type in PII_TYPES:
        sample = pii_data[pii_type][0]
        selected[pii_type] = {
            "pii_type": pii_type,
            "sequence_idx": 0,
            "text_prompt": sample["text_prompt"],
            "target_pii": sample["target_pii"],
        }
    return selected


def tokenize_pii_samples(
    samples: Dict[str, Dict[str, Any]],
    model: HookedTransformer,
    device: str = DEVICE,
    prepend_bos: bool = True,
) -> Dict[str, Dict[str, Any]]:
    """Tokenize prompts and target PII strings for each selected sample.

    For each pii_type, we store:
    - prompt_tokens: [1, prompt_len]
    - target_tokens: [1, target_len]
    - full_tokens: [1, prompt_len + target_len]
    - prompt_len, target_len
    - prompt_str_tokens, target_str_tokens
    """
    tokenized: Dict[str, Dict[str, Any]] = {}

    for pii_type, info in samples.items():
        prompt = info["text_prompt"]
        target = info["target_pii"]

        prompt_tokens: Tensor = model.to_tokens(prompt, prepend_bos=prepend_bos).to(device)
        target_tokens: Tensor = model.to_tokens(target, prepend_bos=False).to(device)

        full_tokens = torch.cat([prompt_tokens, target_tokens], dim=-1).to(device)

        prompt_str_tokens = model.to_str_tokens(prompt, prepend_bos=prepend_bos)
        target_str_tokens = model.to_str_tokens(target, prepend_bos=False)

        tokenized[pii_type] = {
            **info,
            "prompt_tokens": prompt_tokens,
            "target_tokens": target_tokens,
            "full_tokens": full_tokens,
            "prompt_len": prompt_tokens.shape[-1],
            "target_len": target_tokens.shape[-1],
            "prompt_str_tokens": prompt_str_tokens,
            "target_str_tokens": target_str_tokens,
        }

    return tokenized

with PII_DATA_PATH.open("r") as f:
        pii_data = json.load(f)
selected_samples = select_pii_samples(pii_data)
tokenized_samples = tokenize_pii_samples(selected_samples, tl_target_model, device=DEVICE)

for pii_type, sample in tokenized_samples.items():
    print(f"\n=== PII type: {pii_type} ===")
    print(f"Prompt: {sample['text_prompt']}")
    print(f"Target PII: {sample['target_pii']}")
    print(f"Prompt tokens ({sample['prompt_len']}): {sample['prompt_str_tokens']}")
    print(f"Target tokens  (shape: {sample['target_tokens'].shape}): {sample['target_str_tokens']}")
    print(f"Target token ids (shape: {sample['target_tokens'].shape}): {sample['target_tokens']}")
    gen_len = sample["target_len"]
    prompt_tokens = sample["prompt_tokens"]
    if prompt_tokens[0, 0].item() == tl_target_model.tokenizer.bos_token_id:
        gen_prompt_tokens = prompt_tokens[:, 1:]
    else:
        gen_prompt_tokens = prompt_tokens

    with torch.no_grad():
        generated_tokens = tl_target_model.generate(gen_prompt_tokens, max_new_tokens=gen_len, do_sample=False, verbose=False)
        generated_str_tokens = tl_target_model.to_str_tokens(generated_tokens)
        generated_text = tl_target_model.tokenizer.decode(generated_tokens[0])
        # add assertion to make sure that the target pii has been memorized in the target model
        assert sample['target_pii'] in generated_text, f"Target PII not present in generated text: {sample['target_pii']}, generated text: {generated_text}"



Driver's license number of Mahika Ganesh is
tensor([    0,     0, 25603,   434,  7981,  1180,   273, 12828, 11825,   443,
         6597,    73,   310])


IndexError: too many indices for tensor of dimension 2

# Activation Patching Experiments for Memorized PII

The metric we decided to use for evaluating the model performance with activation patching is log-likelihood of the target pii sequence when the model is prompted with a prefix. Since the target pii is composed of multiple consecutive tokens, to get its likelihood from the model we sum the log probabilities of the target pii tokens. If the prompt tokens are $x_{1:\ell}$ and the target PII tokens are $y_{1:N} = (y_1, \dots, y_N)$, then for a model $M$ with token probabilities $p_M(\cdot)$ we define:

\begin{equation*}
\operatorname{LL}_M\big(y_{1:N} \mid x_{1:\ell}\big)
= \sum_{t=1}^N \log p_M\big(y_t \mid x_{1:\ell}, y_{1:t-1}\big).
\end{equation*}

To have a baseline comparison we first compute the log-likelihood of the target pii using the target (memorized model) and the control model

\begin{equation*}
\operatorname{LL}_{\text{target}} = \operatorname{LL}_{M_{\text{target}}}(y_{1:N} \mid x_{1:\ell}),
\qquad
\operatorname{LL}_{\text{control}} = \operatorname{LL}_{M_{\text{control}}}(y_{1:N} \mid x_{1:\ell}).
\end{equation*}

We then define their **baseline improvement**:

\begin{equation*}
\Delta \operatorname{LL}_{\text{base}} = \operatorname{LL}_{\text{target}} - \operatorname{LL}_{\text{control}}.
\end{equation*}

During activation patching, we replace a single layer-and-position activation of the control model with the corresponding activation from the target model. For a given layer $l$ and prompt position $i$, let $\operatorname{LL}_{\text{patched}}^{(l,i)}$ be the log-likelihood of the target PII under this **patched control model**. We then define the **log-likelihood improvement**:

\begin{equation*}
I^{(l,i)} = \operatorname{LL}_{\text{patched}}^{(l,i)} - \operatorname{LL}_{\text{control}}.
\end{equation*}

To normalize the improvement of patching, we divide by the target vs control improvement

\begin{equation*}
\tilde{I}^{(l,i)}
= \frac{I^{(l,i)}}{\Delta \operatorname{LL}_{\text{base}}}
= \frac{\operatorname{LL}_{\text{patched}}^{(l,i)} - \operatorname{LL}_{\text{control}}}{\operatorname{LL}_{\text{target}} - \operatorname{LL}_{\text{control}}}.
\end{equation*}

A value of $\tilde{I}^{(l,i)} \approx 1$ means that patching that specific (layer, position) almost fully recovers the target model's advantage on the memorized PII, highlighting it as an important location for where the PII is stored or routed in the network.

The basis of choosing this metric is from this arxiv article [here](https://arxiv.org/html/2404.15255v1#:~:text=It%E2%80%99s%20easy%20to,in%20this%20section%3A)

The logit difference metric did not seem applicable in our context as we are using the same prompt in both control and target model, and also there is no clear "opposite" answer to the target pii to compute the logit difference against.

## How we do activation patching and teacher-forced evaluation

We run activation patching on the **control** model while always evaluating the log-likelihood of the PII under **teacher forcing**.

Since the target pii is multiple tokens, instead of just supplying the input prompt and then computing the logits for the next token (as done in the ROME paper) we instead use teacher-forcing by supplying the entire sequence (input prompt + target pii), get the metrics by passing that sequence to the model and computing our metric.

- **Full input sequence (prompt + target PII)**  
  For each sample we build a full token sequence
  \begin{equation*}
  \text{full\_tokens} = [x_{1:\ell},\; y_{1:N}],
  \end{equation*}
  where $x_{1:\ell}$ are the prompt tokens and $y_{1:N}$ are the target PII tokens. We feed this entire sequence into both the target and control models.

- **Teacher-forced log-likelihood on the PII span**  
  Given logits over the full sequence, we only evaluate log-probabilities on the **PII targets** tokens:
  \begin{equation*}
  \operatorname{LL}_M\big(y_{1:N} \mid x_{1:\ell}\big)
  = \sum_{t=1}^N \log p_M\big(y_t \mid x_{1:\ell}, y_{1:t-1}\big),
  \end{equation*}

- **Why we use full \(\text{prompt} + \text{PII}\) instead of just the prompt**
  We use the full \(\text{prompt} + \text{PII}\) in order to compute the logits for all the target pii tokens using teacher forcing. Since the model is autoregressive, it only uses information from the past so despite passing the full sequence, we only patch activations related to the input prompt tokens so we do not evaluate activation patching of "future information". We only patch information that the target model has while it is reading the input prompt.

- **How patching is applied**  
  We first run the **target** model once on `full_tokens` and store a cache of activations for every hook point. Then, for a given layer $l$ and prompt position $i$:
  - We run the **control** model on the same `full_tokens`.
  - At the chosen hook and position $i$ in the **prompt**, we **replace** the control activation with the corresponding activation from the target cache. Concretely, in `hook_fn` we do
    \begin{equation*}
    \text{patched}[\;, i, \dots] \leftarrow \text{cached}[\;, i, \dots],
    \end{equation*}
    leaving all other positions and layers untouched.
  - We only patch activations that are at positions of the prompt sequence (not the target pii) and due to the nature of autoregressive models, despite using the full sequence to compute the activations for the target, patching only at positions up to the last prompt token does not lead to using "future knowledge".
  - We then recompute the teacher-forced log-likelihood of the PII under this **patched control model**.

- **Why this makes sense for localization**  
  Because the input tokens are fixed (same `full_tokens`) and we change only a single (layer, position) activation, any change in PII log-likelihood can be attributed to the contribution of that internal site. Sweeping over all layers and prompt positions tells us **where** in the network the target model’s PII-specific behavior is injected into the control model, which is exactly what we want to localize.

In [ ]:
def logprob_from_logits(
    logits: Tensor,
    prompt_len: int,
    target_token_ids: Tensor,
) -> Tuple[float, List[float]]:
    """Compute sequence logprob and per-token logprobs for target tokens.

    Args:
        logits: [1, seq_len, vocab_size] logits for full `prompt + target` sequence.
        prompt_len: Number of tokens in the prompt (including BOS if used).
        target_token_ids: [target_len] tensor of token ids for the target PII.
    """
    assert logits.ndim == 3 and logits.shape[0] == 1, "Only batch size 1 is supported"

    target_token_ids = target_token_ids.to(logits.device)
    target_len = target_token_ids.shape[0]

    # Positions in the sequence where each target token is predicted
    # The logit at position t predicts token at position t+1, so the
    # first target token (at index prompt_len) is predicted from t = prompt_len-1.
    start_pos = prompt_len - 1
    positions = torch.arange(target_len, device=logits.device) + start_pos

    # Slice logits at those positions: [target_len, vocab_size]
    logits_slice = logits[0, positions, :]
    log_probs_slice = torch.log_softmax(logits_slice, dim=-1)

    # Gather logprobs of the actual target tokens
    token_log_probs = log_probs_slice.gather(-1, target_token_ids.unsqueeze(-1)).squeeze(-1)

    sequence_logprob = token_log_probs.sum()
    return float(sequence_logprob.item()), token_log_probs.detach().cpu().tolist()


def logprob_predict(
    model: HookedTransformer,
    prompt_tokens: Tensor,
    target_tokens: Tensor,
) -> Tuple[float, List[float]]:
    """Compute teacher-forced logprob for `target_tokens` given `prompt_tokens`.

    This runs a single forward pass over the full sequence and extracts
    token-level logprobs corresponding to the target PII span.
    """
    full_tokens = torch.cat([prompt_tokens, target_tokens], dim=-1)
    logits = model(full_tokens)

    prompt_len = prompt_tokens.shape[-1]
    target_token_ids = target_tokens[0]

    return logprob_from_logits(logits, prompt_len, target_token_ids)


# Compute baseline metrics for all selected PII samples

baseline_metrics: Dict[str, Dict[str, Any]] = {}

for pii_type, sample in tokenized_samples.items():
    prompt_tokens = sample["prompt_tokens"]
    target_tokens = sample["target_tokens"]

    target_logprob, target_token_logprobs = logprob_predict(
        tl_target_model, prompt_tokens, target_tokens
    )
    control_logprob, control_token_logprobs = logprob_predict(
        tl_control_model, prompt_tokens, target_tokens
    )

    sample["baseline"] = {
        "target_logprob": target_logprob,
        "control_logprob": control_logprob,
        "target_token_logprobs": target_token_logprobs,
        "control_token_logprobs": control_token_logprobs,
    }

    baseline_metrics[pii_type] = {
        "pii_type": pii_type,
        "sequence_idx": sample["sequence_idx"],
        "text_prompt": sample["text_prompt"],
        "target_pii": sample["target_pii"],
        "target_logprob": target_logprob,
        "control_logprob": control_logprob,
        "difference": target_logprob - control_logprob,
    }

baseline_df = pd.DataFrame.from_dict(baseline_metrics, orient="index")
print("\nBaseline metrics (log probabilities of target PII):")
display(baseline_df[["pii_type", "target_logprob", "control_logprob", "difference"]])



Baseline metrics (log probabilities of target PII):


,pii_type,target_logprob,control_logprob,difference
driver_license,driver_license,-12.362307,-59.892303,47.529997
email,email,-10.778866,-39.006397,28.227531
id_number,id_number,-13.094657,-50.170715,37.076058
passport,passport,-13.124870,-43.809155,30.684284


## Layers available to patch
Lets see what layers the model has available for us to patch. Each layer block (number) has the same layers, so lets see each layer hook type and their shape (which are also dependant on the sequence length)

We can see the layer hooks available to use with transformerlens by checking in the cache.keys(). Some hook points are available per layer, some just in the beginning or the end of the model.

In [38]:
# Print out the hook names with their activation shapes
tl_target_model.set_use_attn_result(False)
tl_target_model.set_use_attn_in(False)
tl_target_model.set_use_split_qkv_input(False)
tl_target_model.set_use_hook_mlp_in(False)
_, cache = tl_target_model.run_with_cache("Hey George, how are you doing?", return_cache_object=True)

unique_hooks = set()
for hook_name in cache.keys():
    hook_name_parts = hook_name.split('.')
    if len(hook_name_parts) > 1 and not hook_name_parts[-2].isdigit():
        name = ".".join(hook_name_parts[-2:])
    else:
        name = hook_name_parts[-1]
    print_name = f"{name} ({cache[hook_name].shape})"
    unique_hooks.add(print_name)

print(f"Input tokens: {tl_target_model.to_tokens('Hey George, how are you doing?').shape[-1]}")
hooks = list(unique_hooks)
hooks.sort()
for hook in hooks:
    print(hook)


Input tokens: 9
attn.hook_attn_scores (torch.Size([1, 12, 9, 9]))
attn.hook_k (torch.Size([1, 9, 12, 64]))
attn.hook_pattern (torch.Size([1, 12, 9, 9]))
attn.hook_q (torch.Size([1, 9, 12, 64]))
attn.hook_rot_k (torch.Size([1, 9, 12, 64]))
attn.hook_rot_q (torch.Size([1, 9, 12, 64]))
attn.hook_v (torch.Size([1, 9, 12, 64]))
attn.hook_z (torch.Size([1, 9, 12, 64]))
hook_attn_out (torch.Size([1, 9, 768]))
hook_embed (torch.Size([1, 9, 768]))
hook_mlp_out (torch.Size([1, 9, 768]))
hook_resid_post (torch.Size([1, 9, 768]))
hook_resid_pre (torch.Size([1, 9, 768]))
ln1.hook_normalized (torch.Size([1, 9, 768]))
ln1.hook_scale (torch.Size([1, 9, 1]))
ln2.hook_normalized (torch.Size([1, 9, 768]))
ln2.hook_scale (torch.Size([1, 9, 1]))
ln_final.hook_normalized (torch.Size([1, 9, 768]))
ln_final.hook_scale (torch.Size([1, 9, 1]))
mlp.hook_post (torch.Size([1, 9, 3072]))
mlp.hook_pre (torch.Size([1, 9, 3072]))


In [39]:
tl_utils.get_act_name("mlp_out", 5, "")

'blocks.5.hook_mlp_out'

In [40]:
def hook_fn(activation: Tensor, hook, cache, pos) -> Tensor:
    """Creates the hook function that patches the original activation with the cached activation at the specified position for the hook layer"""
    cached = cache[hook.name]
    patched = activation.clone()
    # Patch only the specified token position
    patched[:, pos, ...] = cached[:, pos, ...]
    return patched

def patched_logprob_for_layer_position(
    control_model: HookedTransformer,
    layer_name: str,
    position_idx: int,
    sample: Dict[str, Any],
    target_cache: Dict[str, Tensor],
    patch_fn = None
) -> float:
    """Compute logprob of target PII when patching one layer+position.

    Args:
        control_model: Control HookedTransformer model to run with patching.
        layer_name: Hook name (e.g., "blocks.3.hook_resid_post").
        position_idx: Token position in the *prompt* sequence to patch.
        sample: Tokenized sample dict (must contain `full_tokens`, `prompt_len`, `target_tokens`).
        target_cache: Cache from the target model for the same `full_tokens`.
    """
    full_tokens: Tensor = sample["full_tokens"]

    logits_patched = control_model.run_with_hooks(
        full_tokens,
        fwd_hooks=[(layer_name, partial(hook_fn, cache=target_cache, pos=position_idx))],
        return_type="logits",
    )

    prompt_len = sample["prompt_len"]
    target_token_ids = sample["target_tokens"][0]

    patched_logprob, _ = logprob_from_logits(logits_patched, prompt_len, target_token_ids)
    return patched_logprob


In [41]:
# Full sweeps over layers and prompt positions (residual, MLP, attention)

num_layers = tl_control_model.cfg.n_layers
num_heads = tl_control_model.cfg.n_heads
tl_control_model.reset_hooks()
tl_target_model.reset_hooks()

def run_component_sweep_for_sample(
    model_to_patch: HookedTransformer,
    model_to_cache: HookedTransformer,
    pii_type: str,
    sample: Dict[str, Any],
    layer_name: str,
    component: str = None,   
) -> Dict[str, Any]:
    """Run a full (layer, position) sweep for a single component type.

    Args:
        pii_type: PII type key.
        sample: Tokenized sample dict.
        component: One of {"residual", "mlp", "attention"}.
        hook_name_fn: Function mapping layer_idx -> hook name.
    """
    prompt_len = sample["prompt_len"]
    control_logprob = sample["baseline"]["control_logprob"]
    target_logprob = sample["baseline"]["target_logprob"]

    # Cache all target activations for this full sequence once
    full_tokens: Tensor = sample["full_tokens"]
    _, cache = model_to_cache.run_with_cache(full_tokens, return_cache_object=True)

    logprob_matrix = np.zeros((prompt_len, num_layers), dtype=np.float32)
    improvement_matrix = np.zeros_like(logprob_matrix)
    normalized_improvement_matrix = np.zeros_like(logprob_matrix)
    records: List[Dict[str, Any]] = []

    for layer_idx in tqdm(range(num_layers), desc=f"{pii_type} {component} {layer_name} layers"):
        debug_log(f"Patching layer {tl_utils.get_act_name(layer_name, layer_idx, component)}")
        for pos in range(prompt_len):  # only positions in the prompt
            patched_logprob = patched_logprob_for_layer_position(
                model_to_patch,
                tl_utils.get_act_name(layer_name, layer_idx, component),
                pos,
                sample,
                cache,
            )
            logprob_matrix[pos, layer_idx] = patched_logprob
            improvement = patched_logprob - control_logprob
            improvement_matrix[pos, layer_idx] = improvement
            normalized_improvement = improvement / (target_logprob - control_logprob)
            normalized_improvement_matrix[pos, layer_idx] = normalized_improvement

            records.append(
                {
                    "pii_type": pii_type,
                    "experiment_type": component,
                    "layer_name": layer_name,
                    "layer_idx": layer_idx,
                    "position_idx": pos,
                    "full_layer_name": tl_utils.get_act_name(layer_name, layer_idx, component),
                    "patched_logprob": patched_logprob,
                    "improvement": improvement,
                    # "success": success,
                }
            )

    results_df = pd.DataFrame(records)
    return {
        "pii_type": pii_type,
        "component": component,
        "logprob_matrix": logprob_matrix,
        "improvement_matrix": improvement_matrix,
        "normalized_improvement_matrix": normalized_improvement_matrix,
        "results_df": results_df,
        "position_tokens": sample["prompt_str_tokens"],
        "control_logprob": control_logprob,
        "target_logprob": sample["baseline"]["target_logprob"],
        "target_pii": sample["target_pii"],
    }


# Run sweeps for all PII types and components
sweep_results: Dict[str, Dict[str, Dict[str, Any]]] = {}

for pii_type, sample in tokenized_samples.items():
    print(f"\n=== Running sweeps for PII type: {pii_type} ===")
    sweep_results[pii_type] = {}
    sweep_results[pii_type]["residual"] = run_component_sweep_for_sample(
        tl_control_model, tl_target_model, pii_type, sample, "post", "res"
    )
    sweep_results[pii_type]["mlp-post"] = run_component_sweep_for_sample(
        tl_control_model, tl_target_model, pii_type, sample, "post", "mlp"
    )
    sweep_results[pii_type]["mlp_out"] = run_component_sweep_for_sample(
        tl_control_model, tl_target_model, pii_type, sample, "mlp_out"
    )
    sweep_results[pii_type]["attention"] = run_component_sweep_for_sample(
        tl_control_model, tl_target_model, pii_type, sample, "attn_out", ""
    )
print("\nCompleted all sweeps.")



=== Running sweeps for PII type: driver_license ===


driver_license res post layers:   0%|          | 0/12 [00:00<?, ?it/s]

driver_license mlp post layers:   0%|          | 0/12 [00:00<?, ?it/s]

driver_license None mlp_out layers:   0%|          | 0/12 [00:00<?, ?it/s]

driver_license  attn_out layers:   0%|          | 0/12 [00:00<?, ?it/s]

...

passport res post layers:   0%|          | 0/12 [00:00<?, ?it/s]

passport mlp post layers:   0%|          | 0/12 [00:00<?, ?it/s]

passport None mlp_out layers:   0%|          | 0/12 [00:00<?, ?it/s]

passport  attn_out layers:   0%|          | 0/12 [00:00<?, ?it/s]


Completed all sweeps.


In [42]:
TOP_K = 10

for pii_type, comps in sweep_results.items():
    print(f"\n=== Top {TOP_K} locations for PII type: {pii_type} ===")

    for component, result in comps.items():
        df = result["results_df"].copy()

        # Add normalized improvement column for convenience
        baseline_gap = result["target_logprob"] - result["control_logprob"]
        if baseline_gap != 0:
            df["normalized_improvement"] = df["improvement"] / baseline_gap
        else:
            df["normalized_improvement"] = float("nan")

        top_df = df.sort_values("improvement", ascending=True).head(TOP_K)

        print(f"\nComponent: {component}")
        display(
            top_df[
                [
                    "layer_idx",
                    "position_idx",
                    "full_layer_name",
                    "patched_logprob",
                    "improvement",
                    "normalized_improvement",
                ]
            ]
        )



=== Top 10 locations for PII type: driver_license ===

Component: residual


,layer_idx,position_idx,full_layer_name,patched_logprob,improvement,normalized_improvement
55,6,7,blocks.6.mlp.hook_post,-61.217312,-1.325008,-0.027877
31,3,7,blocks.3.mlp.hook_post,-60.783234,-0.890930,-0.018745
95,11,7,blocks.11.mlp.hook_post,-60.719765,-0.827461,-0.017409
20,2,4,blocks.2.mlp.hook_post,-60.606186,-0.713882,-0.015020
11,1,3,blocks.1.mlp.hook_post,-60.573563,-0.681259,-0.014333
39,4,7,blocks.4.mlp.hook_post,-60.442970,-0.550667,-0.011586
28,3,4,blocks.3.mlp.hook_post,-60.401562,-0.509258,-0.010714
27,3,3,blocks.3.mlp.hook_post,-60.396217,-0.503914,-0.010602
35,4,3,blocks.4.mlp.hook_post,-60.362556,-0.470253,-0.009894
47,5,7,blocks.5.mlp.hook_post,-60.348106,-0.455803,-0.009590



Component: mlp-post


,layer_idx,position_idx,full_layer_name,patched_logprob,improvement,normalized_improvement
55,6,7,blocks.6.mlp.hook_post,-61.217312,-1.325008,-0.027877
31,3,7,blocks.3.mlp.hook_post,-60.783234,-0.890930,-0.018745
95,11,7,blocks.11.mlp.hook_post,-60.719765,-0.827461,-0.017409
20,2,4,blocks.2.mlp.hook_post,-60.606186,-0.713882,-0.015020
11,1,3,blocks.1.mlp.hook_post,-60.573563,-0.681259,-0.014333
39,4,7,blocks.4.mlp.hook_post,-60.442970,-0.550667,-0.011586
28,3,4,blocks.3.mlp.hook_post,-60.401562,-0.509258,-0.010714
27,3,3,blocks.3.mlp.hook_post,-60.396217,-0.503914,-0.010602
35,4,3,blocks.4.mlp.hook_post,-60.362556,-0.470253,-0.009894
47,5,7,blocks.5.mlp.hook_post,-60.348106,-0.455803,-0.009590



Component: mlp_out


...

,layer_idx,position_idx,full_layer_name,patched_logprob,improvement,normalized_improvement
17,2,3,blocks.2.mlp.hook_post,-44.472496,-0.663342,-0.021618
24,3,3,blocks.3.mlp.hook_post,-44.292721,-0.483566,-0.015759
12,1,5,blocks.1.mlp.hook_post,-44.103649,-0.294495,-0.009598
4,0,4,blocks.0.mlp.hook_post,-44.091373,-0.282219,-0.009198
8,1,1,blocks.1.mlp.hook_post,-44.083832,-0.274677,-0.008952
62,8,6,blocks.8.mlp.hook_post,-44.029465,-0.220310,-0.007180
21,3,0,blocks.3.mlp.hook_post,-44.010517,-0.201363,-0.006562
53,7,4,blocks.7.mlp.hook_post,-44.008781,-0.199627,-0.006506
15,2,1,blocks.2.mlp.hook_post,-43.997620,-0.188465,-0.006142
23,3,2,blocks.3.mlp.hook_post,-43.981022,-0.171867,-0.005601



Component: mlp_out


,layer_idx,position_idx,full_layer_name,patched_logprob,improvement,normalized_improvement
24,3,3,blocks.3.hook_mlp_out,-44.395412,-0.586258,-0.019106
10,1,3,blocks.1.hook_mlp_out,-44.394886,-0.585732,-0.019089
17,2,3,blocks.2.hook_mlp_out,-44.299099,-0.489944,-0.015967
12,1,5,blocks.1.hook_mlp_out,-44.226395,-0.417240,-0.013598
21,3,0,blocks.3.hook_mlp_out,-44.199173,-0.390018,-0.012711
8,1,1,blocks.1.hook_mlp_out,-44.183136,-0.373981,-0.012188
4,0,4,blocks.0.hook_mlp_out,-44.119869,-0.310715,-0.010126
15,2,1,blocks.2.hook_mlp_out,-44.073425,-0.264271,-0.008613
0,0,0,blocks.0.hook_mlp_out,-44.039665,-0.230511,-0.007512
47,6,5,blocks.6.hook_mlp_out,-43.971512,-0.162357,-0.005291



Component: attention


,layer_idx,position_idx,full_layer_name,patched_logprob,improvement,normalized_improvement
69,9,6,blocks.9.hook_attn_out,-44.783936,-0.974781,-0.031768
55,7,6,blocks.7.hook_attn_out,-44.317822,-0.508667,-0.016577
13,1,6,blocks.1.hook_attn_out,-44.265339,-0.456184,-0.014867
3,0,3,blocks.0.hook_attn_out,-44.243660,-0.434505,-0.014161
48,6,6,blocks.6.hook_attn_out,-44.232597,-0.423443,-0.013800
16,2,2,blocks.2.hook_attn_out,-44.169384,-0.360229,-0.011740
24,3,3,blocks.3.hook_attn_out,-44.150497,-0.341343,-0.011124
30,4,2,blocks.4.hook_attn_out,-44.119362,-0.310207,-0.010110
31,4,3,blocks.4.hook_attn_out,-44.102169,-0.293015,-0.009549
44,6,2,blocks.6.hook_attn_out,-44.101990,-0.292835,-0.009543


In [43]:
# Heatmap visualizations of sweep results using a simple Plotly imshow helper

from typing import Optional, Mapping


def imshow(tensor, **kwargs):
    """Simple wrapper around plotly.express.imshow.

    Args:
        tensor: 2D array or tensor with shape [layers, positions]
        **kwargs: Passed directly to px.imshow (e.g., x, y, labels, title).
    """
    fig = px.imshow(
        tl_utils.to_numpy(tensor),
        color_continuous_midpoint=0.0,
        color_continuous_scale="RdBu",
        **kwargs,
    )
    return fig


# Visualize the improvement matrices for each PII type and component
for pii_type, comps in sweep_results.items():
    for component, result in comps.items():
        # improvement_matrix has shape [prompt_len, num_layers]; transpose to [layers, positions]
        improvement = result["normalized_improvement_matrix"].T

        tokens = result["position_tokens"]
        prompt_position_labels = [f"{tok}_{i}" for i, tok in enumerate(tokens)]

        target_pii = result["target_pii"]

        fig = imshow(
            improvement,
            x=prompt_position_labels,
            title=f"{pii_type} - {component}: patching effect on log-likelihood of target PII",
            labels={
                "x": "Position",
                "y": "Layer",
            },
        )
        os.makedirs(OUTPUT_DIR / f"1_sample", exist_ok=True)
        filepath = OUTPUT_DIR / f"1_sample/impovement_metrics_layer_and_position_patching_{pii_type}_{component}"
        fig.write_html(filepath)
        fig.write_image(filepath.with_suffix(".png"))
        fig.show()
        


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


...

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [44]:
!pip install kaleido

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


## Do Activation Patching across all triger (input prompt) tokens
Since we do not see any effects when doing activation patching on a layer and position level, try a more coarse approach of patching all of the input token activations per layer to see of any stronger effects

In [45]:
# Full sweeps over layers and prompt positions (residual, MLP, attention)

num_layers = tl_control_model.cfg.n_layers
num_heads = tl_control_model.cfg.n_heads
tl_control_model.reset_hooks()
tl_target_model.reset_hooks()


# Patch all tokens up to and including `last_pos`
def hook_fn_prefix(activation: Tensor, hook, cache, last_pos: int) -> Tensor:
    cached = cache[hook.name]
    patched = activation.clone()
    patched[:, : last_pos + 1, ...] = cached[:, : last_pos + 1, ...]
    return patched


def patched_logprob_for_layer_up_to_position(
    model: HookedTransformer,
    layer_name: str,
    sample: Dict[str, Any],
    cache: Dict[str, Tensor],
) -> float:
    """Compute logprob of target PII when patching a layer on all prompt positions."""
    full_tokens: Tensor = sample["full_tokens"]
    prompt_len = sample["prompt_len"]          # last prompt index = prompt_len - 1

    logits_patched = model.run_with_hooks(
        full_tokens,
        fwd_hooks=[(
            layer_name,
            partial(hook_fn_prefix, cache=cache, last_pos=prompt_len - 1),
        )],
        return_type="logits",
    )

    target_token_ids = sample["target_tokens"][0]
    patched_logprob, _ = logprob_from_logits(logits_patched, prompt_len, target_token_ids)
    return patched_logprob

def run_component_sweep_for_sample_all_input_tokens_pos(
    model_to_patch: HookedTransformer,
    model_to_cache: HookedTransformer,
    pii_type: str,
    sample: Dict[str, Any],
    layer_name: str,
    component: str = None,
) -> Dict[str, Any]:
    prompt_len = sample["prompt_len"]
    control_logprob = sample["baseline"]["control_logprob"]
    target_logprob = sample["baseline"]["target_logprob"]

    full_tokens: Tensor = sample["full_tokens"]
    _, cache = model_to_cache.run_with_cache(full_tokens, return_cache_object=True)

    layer_logprobs = np.zeros(num_layers, dtype=np.float32)
    layer_improvements = np.zeros_like(layer_logprobs)
    layer_norm_improvements = np.zeros_like(layer_logprobs)
    records: List[Dict[str, Any]] = []

    for layer_idx in tqdm(range(num_layers), desc=f"{pii_type} {component} {layer_name} layers"):
        full_layer_name = tl_utils.get_act_name(layer_name, layer_idx, component)
        debug_log(f"Patching layer {full_layer_name} on all prompt positions")

        patched_logprob = patched_logprob_for_layer_up_to_position(
            model_to_patch,
            full_layer_name,
            sample,
            cache,
        )

        improvement = patched_logprob - control_logprob
        norm_improvement = improvement / (target_logprob - control_logprob)

        layer_logprobs[layer_idx] = patched_logprob
        layer_improvements[layer_idx] = improvement
        layer_norm_improvements[layer_idx] = norm_improvement

        records.append(
            {
                "pii_type": pii_type,
                "experiment_type": component,
                "layer_name": layer_name,
                "layer_idx": layer_idx,
                "position_idx": prompt_len - 1,  # last prompt position (for bookkeeping)
                "full_layer_name": full_layer_name,
                "patched_logprob": patched_logprob,
                "improvement": improvement,
                "normalized_improvement": norm_improvement,
            }
        )

    results_df = pd.DataFrame(records)
    return {
        "pii_type": pii_type,
        "component": component,
        "layer_logprobs": layer_logprobs,
        "layer_improvements": layer_improvements,
        "layer_normalized_improvements": layer_norm_improvements,
        "results_df": results_df,
        "position_tokens": sample["prompt_str_tokens"],
        "control_logprob": control_logprob,
        "target_logprob": target_logprob,
        "target_pii": sample["target_pii"],
    }


# Run sweeps for all PII types and components
sweep_results_all_input_tokens_pos: Dict[str, Dict[str, Dict[str, Any]]] = {}

for pii_type, sample in tokenized_samples.items():
    print(f"\n=== Running sweeps for PII type: {pii_type} ===")
    sweep_results_all_input_tokens_pos[pii_type] = {}
    sweep_results_all_input_tokens_pos[pii_type]["residual"] = run_component_sweep_for_sample_all_input_tokens_pos(
        tl_control_model, tl_target_model, pii_type, sample, "post", "res"
    )
    sweep_results_all_input_tokens_pos[pii_type]["mlp-post"] = run_component_sweep_for_sample_all_input_tokens_pos(
        tl_control_model, tl_target_model, pii_type, sample, "post", "mlp"
    )
    sweep_results_all_input_tokens_pos[pii_type]["mlp_out"] = run_component_sweep_for_sample_all_input_tokens_pos(
        tl_control_model, tl_target_model, pii_type, sample, "mlp_out"
    )
    sweep_results_all_input_tokens_pos[pii_type]["attention"] = run_component_sweep_for_sample_all_input_tokens_pos(
        tl_control_model, tl_target_model, pii_type, sample, "attn_out", ""
    )
print("\nCompleted all sweeps.")


=== Running sweeps for PII type: driver_license ===


driver_license res post layers:   0%|          | 0/12 [00:00<?, ?it/s]

driver_license mlp post layers:   0%|          | 0/12 [00:00<?, ?it/s]

driver_license None mlp_out layers:   0%|          | 0/12 [00:00<?, ?it/s]

driver_license  attn_out layers:   0%|          | 0/12 [00:00<?, ?it/s]

...

passport res post layers:   0%|          | 0/12 [00:00<?, ?it/s]

passport mlp post layers:   0%|          | 0/12 [00:00<?, ?it/s]

passport None mlp_out layers:   0%|          | 0/12 [00:00<?, ?it/s]

passport  attn_out layers:   0%|          | 0/12 [00:00<?, ?it/s]


Completed all sweeps.


In [46]:
import numpy as np
import plotly.express as px

for pii_type, comps in sweep_results_all_input_tokens_pos.items():
    for component, result in comps.items():
        layer_norm = result["layer_normalized_improvements"]  # shape [num_layers]
        num_layers = len(layer_norm)

        fig = px.line(
            x=np.arange(num_layers),
            y=layer_norm,
            markers=True,
            title=f"{pii_type} - {component}: normalized improvement by layer",
            labels={
                "x": "Layer index",
                "y": "Normalized Δ log-sum p(target_pii)",
            },
        )
        fig.show()
        os.makedirs(OUTPUT_DIR / f"all_input_tokens_pos", exist_ok=True)
        path = OUTPUT_DIR / f"all_input_tokens_pos" / f"{pii_type}_{component}_all_input_tokens_pos.png"
        fig.write_image(f"{pii_type}_{component}_all_input_tokens_pos.png")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


...

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


### Patching from control --> target model

In [47]:
# lets try patching from the control to the target model to see which layers and positions would lead to the target model performing worse on the pii
# Run sweeps for all PII types and components
control_to_target_sweep_results: Dict[str, Dict[str, Dict[str, Any]]] = {}

for pii_type, sample in tokenized_samples.items():
    print(f"\n=== Running sweeps for PII type: {pii_type} ===")
    control_to_target_sweep_results[pii_type] = {}
    control_to_target_sweep_results[pii_type]["residual"] = run_component_sweep_for_sample(
        tl_target_model, tl_control_model, pii_type, sample, "post", "res"
    )
    control_to_target_sweep_results[pii_type]["mlp-post"] = run_component_sweep_for_sample(
        tl_target_model, tl_control_model, pii_type, sample, "post", "mlp"
    )
    control_to_target_sweep_results[pii_type]["mlp_out"] = run_component_sweep_for_sample(
        tl_target_model, tl_control_model, pii_type, sample, "mlp_out"
    )
    control_to_target_sweep_results[pii_type]["attention"] = run_component_sweep_for_sample(
        tl_target_model, tl_control_model, pii_type, sample, "attn_out", ""
    )
print("\nCompleted all sweeps.")




=== Running sweeps for PII type: driver_license ===


driver_license res post layers:   0%|          | 0/12 [00:00<?, ?it/s]

driver_license mlp post layers:   0%|          | 0/12 [00:00<?, ?it/s]

driver_license None mlp_out layers:   0%|          | 0/12 [00:00<?, ?it/s]

driver_license  attn_out layers:   0%|          | 0/12 [00:00<?, ?it/s]

...

passport res post layers:   0%|          | 0/12 [00:00<?, ?it/s]

passport mlp post layers:   0%|          | 0/12 [00:00<?, ?it/s]

passport None mlp_out layers:   0%|          | 0/12 [00:00<?, ?it/s]

passport  attn_out layers:   0%|          | 0/12 [00:00<?, ?it/s]


Completed all sweeps.


In [48]:
# Visualize the improvement matrices for each PII type and component
for pii_type, comps in control_to_target_sweep_results.items():
    for component, result in comps.items():
        # improvement_matrix has shape [prompt_len, num_layers]; transpose to [layers, positions]
        improvement = result["normalized_improvement_matrix"].T

        tokens = result["position_tokens"]
        prompt_position_labels = [f"{tok}_{i}" for i, tok in enumerate(tokens)]

        target_pii = result["target_pii"]

        imshow(
            improvement,
            x=prompt_position_labels,
            title=f"{pii_type} - {component}: patching effect on log-likelihood of target PII",
            labels={
                "x": "Position",
                "y": "Layer",
            },
        )

patching from the control to the target model we see again no real change in the prediction (most layer positions still have 1 value) so patching certain layers in the target model does not influence the behaviour

# Patch Multiple Activations

In [49]:
num_layers = tl_control_model.cfg.n_layers

def hook_fn_all_positions(activation: Tensor, hook, cache, last_pos: int) -> Tensor:
    cached = cache[hook.name]
    patched = activation.clone()
    patched[:, : last_pos + 1, ...] = cached[:, : last_pos + 1, ...]
    return patched


def patched_logprob_multiple_layers(
    model_to_patch: HookedTransformer,
    model_to_cache: HookedTransformer,
    sample: Dict[str, Any],
    layer_names: List[str],
) -> float:
    full_tokens: Tensor = sample["full_tokens"]
    prompt_len = sample["prompt_len"]
    target_logprob = sample["baseline"]["target_logprob"]
    control_logprob = sample["baseline"]["control_logprob"]

    _, cache = model_to_cache.run_with_cache(full_tokens, return_cache_object=True)

    logits_patched = model_to_patch.run_with_hooks(
        full_tokens,
        fwd_hooks=[(
            layer_name,
            partial(hook_fn_prefix, cache=cache, last_pos=prompt_len - 1),
        ) for layer_name in layer_names],
        return_type="logits",
    )

    target_token_ids = sample["target_tokens"][0]
    patched_logprob, _ = logprob_from_logits(logits_patched, prompt_len, target_token_ids)
    
    improvement = patched_logprob - control_logprob
    norm_improvement = improvement / (target_logprob - control_logprob)

    return patched_logprob, norm_improvement


In [32]:
for pii_type, sample in tokenized_samples.items():
    print(f"\n=== Running sweeps for PII type: {pii_type} ===")
    print("resid_post")
    layer_names = [tl_utils.get_act_name("resid_post", layer_idx, "") for layer_idx in range(num_layers)]
    print(layer_names)
    p, n = patched_logprob_multiple_layers(tl_control_model, tl_target_model, tokenized_samples[pii_type], layer_names)
    print(f"baseline logprob: {tokenized_samples[pii_type]['baseline']['control_logprob']}")
    print(f"target logprob: {tokenized_samples[pii_type]['baseline']['target_logprob']}")
    print(f"patched_logprob: {p}, norm_improvement: {n}")

    print("mlp_out")
    layer_names = [tl_utils.get_act_name("mlp_out", layer_idx, "") for layer_idx in range(num_layers)]
    print(layer_names)
    p, n = patched_logprob_multiple_layers(tl_control_model, tl_target_model, tokenized_samples[pii_type], layer_names)
    print(f"baseline logprob: {tokenized_samples[pii_type]['baseline']['control_logprob']}")
    print(f"target logprob: {tokenized_samples[pii_type]['baseline']['target_logprob']}")
    print(f"patched_logprob: {p}, norm_improvement: {n}")

    print("attn_out")
    layer_names = [tl_utils.get_act_name("attn_out", layer_idx, "") for layer_idx in range(num_layers)]
    print(layer_names)
    p, n = patched_logprob_multiple_layers(tl_control_model, tl_target_model, tokenized_samples[pii_type], layer_names)
    print(f"baseline logprob: {tokenized_samples[pii_type]['baseline']['control_logprob']}")
    print(f"target logprob: {tokenized_samples[pii_type]['baseline']['target_logprob']}")
    print(f"patched_logprob: {p}, norm_improvement: {n}")



=== Running sweeps for PII type: driver_license ===
resid_post
['blocks.0.hook_resid_post', 'blocks.1.hook_resid_post', 'blocks.2.hook_resid_post', 'blocks.3.hook_resid_post', 'blocks.4.hook_resid_post', 'blocks.5.hook_resid_post', 'blocks.6.hook_resid_post', 'blocks.7.hook_resid_post', 'blocks.8.hook_resid_post', 'blocks.9.hook_resid_post', 'blocks.10.hook_resid_post', 'blocks.11.hook_resid_post']
baseline logprob: -59.892303466796875
target logprob: -12.362306594848633
patched_logprob: -56.32792282104492, norm_improvement: 0.07499223396447596
mlp_out
['blocks.0.hook_mlp_out', 'blocks.1.hook_mlp_out', 'blocks.2.hook_mlp_out', 'blocks.3.hook_mlp_out', 'blocks.4.hook_mlp_out', 'blocks.5.hook_mlp_out', 'blocks.6.hook_mlp_out', 'blocks.7.hook_mlp_out', 'blocks.8.hook_mlp_out', 'blocks.9.hook_mlp_out', 'blocks.10.hook_mlp_out', 'blocks.11.hook_mlp_out']
baseline logprob: -59.892303466796875
target logprob: -12.362306594848633
patched_logprob: -62.00569152832031, norm_improvement: -0.04446

## Summary and Usage

This notebook implements the full activation patching pipeline described in `experiment-design.md`:

- Loads target/control Pythia models into TransformerLens and validates them
- Loads memorized PII data, selects one sample per PII type, and computes baseline teacher-forced logprobs
- Runs full residual/MLP/attention sweeps over all layers and prompt positions and saves results/heatmaps
- Compares MLP vs attention at important (layer, position) locations and performs head-level attention patching

Results (matrices, CSVs, JSON metadata, and plots) are written under `activation_patching_results/{MODEL_SIZE}M/` in the structure specified in the experimental design.

